In [1]:
from django.forms.models import model_to_dict
from db.models import *
from datetime import datetime

In [2]:
def clear():
    User.objects.all().delete()
    Blog.objects.all().delete()
    Topic.objects.all().delete()

In [3]:
clear()

In [4]:
def create():
    u1 = User(first_name='u1', last_name='u1')
    u2 = User(first_name='u2', last_name='u2')
    u3 = User(first_name='u3', last_name='u3')

    u1.save()
    u2.save()
    u3.save()

    blog1 = Blog(title='blog1', author=u1)
    blog2 = Blog(title='blog2', author=u1)
    blog1.save()
    blog2.save()
    blog1.subscribers.add(u1, u2)
    blog2.subscribers.add(u2)

    topic1 = Topic(title='topic1', blog=blog1, author=u1)
    topic2 = Topic(title='topic2_content', blog=blog1, author=u3, created='2017-01-01')
    topic1.save()
    topic2.save()
    topic1.likes.add(u1, u2, u3)

In [5]:
create()

C:\Users\LyginYuALaptop\.virtualenvs\python-3.6\lib\site-packages\django\db\models\fields\__init__.py:1447: RuntimeWarning: DateTimeField Topic.created received a naive datetime (2017-01-01 00:00:00) while time zone support is active.
  RuntimeWarning)


In [6]:
User.objects.all()

<QuerySet [<User: 52,u1,u1>, <User: 53,u2,u2>, <User: 54,u3,u3>]>

In [7]:
Blog.objects.all()

<QuerySet [<Blog: 35,blog1,52>, <Blog: 36,blog2,52>]>

In [8]:
Topic.objects.all()

<QuerySet [<Topic: 35,topic1,35,52,2019-10-31 07:04:43.069360+00:00,db.User.None>, <Topic: 36,topic2_content,35,54,2017-01-01 00:00:00+00:00,db.User.None>]>

In [9]:
def edit_all():
    for user in User.objects.all():
        user.first_name = 'uu1'
        user.save()

In [10]:
edit_all()

In [11]:
def edit_u1_u2():
    for user in User.objects.filter(Q(first_name='u1') | Q(first_name='u2')):
        user.first_name = 'uu1'
        user.save()

In [12]:
edit_u1_u2()

In [13]:
def delete_u1():
    for user in User.objects.filter(first_name='u1'):
        user.delete()

In [14]:
delete_u1()

In [15]:
def unsubscribe_u2_from_blogs():
    for user in User.objects.filter(first_name='u2'):
        for blog in Blog.objects.all():
            blog.subscribers.remove(user)

In [16]:
unsubscribe_u2_from_blogs()

In [17]:
def get_topic_created_grated():
    return Topic.objects.filter(created__gt='2018-01-01')

In [18]:
get_topic_created_grated()

C:\Users\LyginYuALaptop\.virtualenvs\python-3.6\lib\site-packages\django\db\models\fields\__init__.py:1447: RuntimeWarning: DateTimeField Topic.created received a naive datetime (2018-01-01 00:00:00) while time zone support is active.
  RuntimeWarning)


<QuerySet [<Topic: 35,topic1,35,52,2019-10-31 07:04:43.069360+00:00,db.User.None>]>

In [19]:
def get_topic_title_ended():
    return Topic.objects.filter(title__endswith='content')

In [20]:
get_topic_title_ended()

<QuerySet [<Topic: 36,topic2_content,35,54,2017-01-01 00:00:00+00:00,db.User.None>]>

In [21]:
def get_user_with_limit():
    return User.objects.all().order_by('-id')[:2]

In [22]:
get_user_with_limit()

<QuerySet [<User: 54,uu1,u3>, <User: 53,uu1,u2>]>

In [45]:
def get_topic_count():
    return Blog.objects.annotate(topic_count=Count('topic')).order_by('topic_count')

In [50]:
get_topic_count()

<QuerySet [<Blog: 36,blog2,52>, <Blog: 35,blog1,52>]>

In [44]:
print(Blog.objects.annotate(topic_count=Count('topic')).order_by('topic_count').query)

SELECT "db_blog"."id", "db_blog"."title", "db_blog"."author_id", "db_blog"."created", COUNT("db_topic"."id") AS "topic_count" FROM "db_blog" LEFT OUTER JOIN "db_topic" ON ("db_blog"."id" = "db_topic"."blog_id") GROUP BY "db_blog"."id", "db_blog"."title", "db_blog"."author_id", "db_blog"."created" ORDER BY "topic_count" ASC


In [25]:
print(Blog.objects.annotate(topic_count=Count('topic')).query)

SELECT "db_blog"."id", "db_blog"."title", "db_blog"."author_id", "db_blog"."created", COUNT("db_topic"."id") AS "topic_count" FROM "db_blog" LEFT OUTER JOIN "db_topic" ON ("db_blog"."id" = "db_topic"."blog_id") GROUP BY "db_blog"."id", "db_blog"."title", "db_blog"."author_id", "db_blog"."created"


In [73]:
def get_avg_topic_count():
    return Blog.objects.annotate(topic_count=Count('topic')).aggregate(avg=Avg('topic_count'))

In [54]:
Blog.objects.annotate(count=Count('topic')).aggregate(Avg('count'))

{'count__avg': 1.0}

In [72]:
Blog.objects.aggregate(Count('author'))

{'author__count': 2}

In [74]:
print(Blog.objects.annotate(topic_count=Count('topic')).aggregate(avg=Avg('topic_count')))

{'avg': 1.0}


In [68]:
print(Blog.objects.annotate(count=Count('topic')).query)

SELECT "db_blog"."id", "db_blog"."title", "db_blog"."author_id", "db_blog"."created", COUNT("db_topic"."id") AS "count" FROM "db_blog" LEFT OUTER JOIN "db_topic" ON ("db_blog"."id" = "db_topic"."blog_id") GROUP BY "db_blog"."id", "db_blog"."title", "db_blog"."author_id", "db_blog"."created"


In [29]:
def get_blog_that_have_more_than_one_topic():
    return Blog.objects.annotate(topic_count=Count('topic')).filter(topic_count__gt=1)

In [30]:
get_blog_that_have_more_than_one_topic()

<QuerySet [<Blog: 35,blog1,52>]>

In [31]:
print(Blog.objects.annotate(topic_count=Count('topic')).filter(topic_count__gt=1).query)

SELECT "db_blog"."id", "db_blog"."title", "db_blog"."author_id", "db_blog"."created", COUNT("db_topic"."id") AS "topic_count" FROM "db_blog" LEFT OUTER JOIN "db_topic" ON ("db_blog"."id" = "db_topic"."blog_id") GROUP BY "db_blog"."id", "db_blog"."title", "db_blog"."author_id", "db_blog"."created" HAVING COUNT("db_topic"."id") > 1


In [32]:
def get_topic_by_u1():
    return Topic.objects.filter(author__first_name='u1')

In [33]:
get_topic_by_u1()

<QuerySet []>

In [34]:
def get_user_that_dont_have_blog():
    return User.objects.filter(blog__author__isnull=True)

In [35]:
User.objects.filter(blog__author__isnull=True)

<QuerySet [<User: 53,uu1,u2>, <User: 54,uu1,u3>]>

In [36]:
User.objects.filter(blog__author__isnull=True, blog__isnull=False)

<QuerySet []>

In [37]:
print(User.objects.filter(blog__author__isnull=True).query)

SELECT "db_user"."id", "db_user"."first_name", "db_user"."last_name" FROM "db_user" LEFT OUTER JOIN "db_blog" ON ("db_user"."id" = "db_blog"."author_id") WHERE "db_blog"."author_id" IS NULL


In [96]:
def get_topic_that_like_all_users():
    amount_of_users = User.objects.all().aggregate(count=Count('pk'))['count']
    return Topic.objects.annotate(count_likes_user=Count('likes')).filter(count_likes_user=amount_of_users)

In [97]:
get_topic_that_like_all_users()

<QuerySet [<Topic: 35,topic1,35,52,2019-10-31 07:04:43.069360+00:00,db.User.None>]>

In [85]:
amount_of_users = User.objects.all().aggregate(count=Count('pk'))['count']

In [93]:
print(Topic.objects.annotate(count_likes_user=Count('likes')).filter(count_likes_user=amount_of_users).query)

SELECT "db_topic"."id", "db_topic"."title", "db_topic"."blog_id", "db_topic"."author_id", "db_topic"."created", COUNT("db_topic_likes"."user_id") AS "count_likes_user" FROM "db_topic" LEFT OUTER JOIN "db_topic_likes" ON ("db_topic"."id" = "db_topic_likes"."topic_id") GROUP BY "db_topic"."id", "db_topic"."title", "db_topic"."blog_id", "db_topic"."author_id", "db_topic"."created" HAVING COUNT("db_topic_likes"."user_id") = 3


In [98]:
def get_topic_that_dont_have_like():
    return Topic.objects.annotate(count_likes_user=Count('likes')).filter(count_likes_user=0)

In [99]:
print(Topic.objects.annotate(count_likes_user=Count('likes')).filter(count_likes_user=0).query)

SELECT "db_topic"."id", "db_topic"."title", "db_topic"."blog_id", "db_topic"."author_id", "db_topic"."created", COUNT("db_topic_likes"."user_id") AS "count_likes_user" FROM "db_topic" LEFT OUTER JOIN "db_topic_likes" ON ("db_topic"."id" = "db_topic_likes"."topic_id") GROUP BY "db_topic"."id", "db_topic"."title", "db_topic"."blog_id", "db_topic"."author_id", "db_topic"."created" HAVING COUNT("db_topic_likes"."user_id") = 0
